In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms

In [24]:
model = models.resnet18(pretrained=True)

c:\Users\tsfai\Projects\data-slayer-2.0\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\tsfai\Projects\data-slayer-2.0\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [25]:
num_features = model.fc.in_features
num_features

512

In [26]:
model.fc = nn.Sequential(
    nn.Linear(num_features, 1),
    nn.Sigmoid()
)

In [8]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [75]:
from torch.utils.data import Dataset
from PIL import Image
import pandas as pd
import os

In [32]:
data_train = pd.read_csv("../data/data_train_f.csv")
data_train.head()

,path,label,conf,revalidated_manually
0,.\data\train\subject-1\fall\1_backward_falls\f...,0,0.000000,1
1,.\data\train\subject-1\fall\1_backward_falls\f...,0,0.000000,1
2,.\data\train\subject-1\fall\1_backward_falls\f...,1,0.444923,1
3,.\data\train\subject-1\fall\1_backward_falls\f...,1,0.616470,1
4,.\data\train\subject-1\fall\1_backward_falls\f...,0,0.000000,1


In [84]:
class CustomImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths 
        self.labels = labels
        self.transform = transform

        for i in range(len(self.image_paths)):
            if self.labels is None:
                self.image_paths[i] = os.path.join("..", "data", "test", self.image_paths[i])
            else:
                self.image_paths[i] = "..\\" + self.image_paths[i]

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        if self.labels is None:
            return image
        else:
            label = self.labels[idx]

        return image, label

In [34]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(data_train, test_size=0.2, random_state=42)

In [35]:
train_data_paths = train_data['path'].values.tolist()
val_data_paths = val_data['path'].values.tolist()

train_data_labels = train_data['label'].values.tolist()
val_data_labels = val_data['label'].values.tolist()

train_dataset = CustomImageDataset(image_paths=train_data_paths, labels=train_data_labels, transform=transform)
val_dataset = CustomImageDataset(image_paths=val_data_paths, labels=val_data_labels, transform=transform)

In [36]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [7]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

next(model.parameters()).device

device(type='cuda', index=0)

In [54]:
# for inputs, labels in train_loader:
#     print(inputs.shape, labels.shape)
#     outputs = model(inputs)
#     print(outputs.min(), outputs.max())
#     break

In [62]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        labels = labels.unsqueeze(1).float()

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total * 100
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}")

    if (epoch + 1) % 5 == 0:
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                predicted = (outputs > 0.5).long()
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        test_acc = correct / total * 100
        print(f"Test Accuracy: {test_acc:.2f}%")


Epoch 1/10, Loss: 0.0450, Accuracy: 98.60
Epoch 2/10, Loss: 0.0161, Accuracy: 99.39
Epoch 3/10, Loss: 0.0319, Accuracy: 98.95
Epoch 4/10, Loss: 0.0065, Accuracy: 99.83
Epoch 5/10, Loss: 0.0215, Accuracy: 99.45
Test Accuracy: 898.37%
Epoch 6/10, Loss: 0.0142, Accuracy: 99.53
Epoch 7/10, Loss: 0.0091, Accuracy: 99.77
Epoch 8/10, Loss: 0.0040, Accuracy: 99.85
Epoch 9/10, Loss: 0.0041, Accuracy: 99.91
Epoch 10/10, Loss: 0.0029, Accuracy: 99.91
Test Accuracy: 897.67%


In [63]:
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss 
}, "../weights/checkpoints/001.pth")

In [64]:
torch.save(model.state_dict(), "../weights/weights_001.pth")

In [5]:
checkpoint = torch.load("../weights/checkpoints/001.pth")

C:\Users\tsfai\AppData\Local\Temp\ipykernel_31964\1938824417.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("../weights/checkpoints/001.pth")


In [29]:
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

epoch = checkpoint['epoch']
loss = checkpoint['loss']

print(f"Model loaded from epoch {epoch} with training loss {loss:.4f}")

# Set Evaluation Mode
model.eval()

Model loaded from epoch 9 with training loss 0.0004


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [22]:
import torch.nn.functional as F

In [46]:
def evaluate_model(model, dataloader, criterion):
    model.eval()
    total_loss = .0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.unsqueeze(1).float()

            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            preds = (outputs > 0.5).long()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    
    avg_loss = total_loss / len(dataloader)
    accuracy = correct / total
    
    return avg_loss, accuracy

In [47]:
train_loss, train_acc = evaluate_model(model, train_loader, criterion)
val_loss, val_acc = evaluate_model(model, val_loader, criterion)

print(f"Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.4f}")
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

Training Loss: 0.0112, Training Accuracy: 0.9985
Validation Loss: 0.0418, Validation Accuracy: 0.9930


K-Fold Validation!!!!!!

In [48]:
from sklearn.model_selection import KFold

In [49]:
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

In [50]:
fold_train_losses = []
fold_train_accuracies = []
fold_val_losses = []
fold_val_accuracies = []

In [ ]:
# for fold, (train_indices, val_indices) in enumerate(kf.split(data_train)):
#     print(f"Fold {fold+1}/{k_folds}")

#     train_subset = CustomImageDataset(data_train)

Attempt Submission

In [85]:
test_data = pd.DataFrame({
    "id": os.listdir("../data/test")
})

test_data.head()

,id
0,00396960ae.jpg
1,004773d4e8.jpg
2,009b266d98.jpg
3,009ed56c89.jpg
4,00a16cb3ae.jpg


In [86]:
test_dataset = CustomImageDataset(image_paths=test_data['id'].values.tolist(), labels=None, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [88]:
model.eval()
predictions = []

with torch.no_grad():
    for images in test_loader:
        images = images.to(device)
        outputs = model(images)

        preds = (outputs > 0.5).long()
        predictions.extend(preds.cpu().numpy())

In [92]:
predictions = [pred[0] for pred in predictions]
predictions

[np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(0),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.in

In [93]:
test_data['label'] = predictions
test_data.head()

,id,label
0,00396960ae.jpg,0
1,004773d4e8.jpg,0
2,009b266d98.jpg,1
3,009ed56c89.jpg,0
4,00a16cb3ae.jpg,0


In [95]:
test_data.to_csv("../data/submissions/001.csv", index=False)